In [ ]:
import diagonal_b6 as b6
import pandas as pd

In [ ]:
# connect to the world
grpc_address = "localhost:8002"
w = b6.connect_insecure(grpc_address)

In [ ]:
OAKLEY_SQUARE_GARDENS_WAY_ID = 26717682
OAKLEY_SQUARE_GARDENS_BENCH_NODE_ID = 2345394864
OAKLEY_SQUARE_GARDENS_LATLON = (51.5341468, -0.1361648)
OAKLEY_SQUARE_GARDENS_POLYGON_WKT = "POLYGON ((-0.13570649999999998 51.53506949999999, -0.1357177 51.53505379999999, -0.13576050000000006 51.5349936, -0.13578839999999998 51.5349543, -0.13609400000000002 51.5345244, -0.13676459999999996 51.533692499999994, -0.1367822 51.533670699999995, -0.1370947 51.533283, -0.13707020000000003 51.5332622, -0.1370397 51.53323629999999, -0.13676499999999997 51.53337659999999, -0.1364471 51.533537399999986, -0.1364129 51.533554699999996, -0.1362849 51.53361949999999, -0.13611429999999997 51.5337814, -0.13570090000000004 51.53428509999999, -0.1355705 51.534519, -0.13555600000000004 51.53494479999999, -0.1355552 51.5349682, -0.13555399999999998 51.5350039, -0.13555180000000003 51.5350676, -0.1355503 51.53511269999999, -0.13570649999999998 51.53506949999999))"

## The world?
One of the huge benefits of bedrock is that it allows you to build up a query or analysis without evaluating it. 

You evaluate your `query`, for example, via applying `w` to it. 

You will see this across the examples as `w(query)`

If you want to export results out of geojson, you can use `w(b6.to_geojson_collection(query)))` if its a collection of features, or `w(b6.to_geojson(query)))` if its just one feature.

### a note about OSM
Our default world loads OSM data, we can represent different datasets but handle OSM well. As such - all the examples below deal with OSM data. 


## Tags

`b6` lets you search the world in whatever data you have loaded. 
Since by default we have loaded in OSM, you do this via OSM tags (https://wiki.openstreetmap.org/wiki/Tags). Not all tags are "searchable", the ones that are have a `#` up front. Here are ones we have made searchable by default:

	"amenity":   "#amenity"
	"barrier":   "#barrier"
	"boundary":  "#boundary"
	"bridge":    "#bridge"
	"building":  "#building"
	"highway":   "#highway"
	"landuse":   "#landuse"
	"leisure":   "#leisure"
	"natural":   "#natural"
	"network":   "#network"
	"place":     "#place"
	"railway":   "#railway"
	"route":     "#route"
	"shop":      "#shop"
	"water":     "#water"
	"waterway":  "#waterway"
	"fhrs:id":   "@fhrs:id"
	"wikidata":  "@wikidata"
	"wikipedia": "@wikipedia"

In [ ]:
# find all the parks - https://wiki.openstreetmap.org/wiki/Tag:leisure%3Dpark
parks = b6.find( b6.tagged("#leisure","park") )

# access the metadata via `all_tags()`
for osmid,park in w(parks):
    print( f"{osmid}\n{park.all_tags()}" )

print("----")

# export the set of features as a geojson
geojson = w(b6.to_geojson_collection(parks))
print(geojson)

In [ ]:
# you could also generically find all the features within the "leisure" key
# https://wiki.openstreetmap.org/wiki/Key:leisure
leisure_features = b6.find( b6.keyed("#leisure") )

# access the metadata via `all_tags()`
for osmid,feature in w(leisure_features):
    print( f"{osmid}\n{feature.all_tags()}" )

## OSM IDs
`b6` is able to find features directly from their `OSM ID`

In [ ]:
# or if you knew the OSM ID of a park you could find it that way
oakleysq_gardens = b6.find_area( b6.osm_way_area_id(OAKLEY_SQUARE_GARDENS_WAY_ID) )

print( w(oakleysq_gardens).all_tags() )

print("----")

# export the feature as a geojson
geojson = w(b6.to_geojson(oakleysq_gardens))
print(geojson)

In [ ]:
# you can also search for a bench in the park
bench = b6.find_point( b6.osm_node_id(OAKLEY_SQUARE_GARDENS_BENCH_NODE_ID ) )

print( w(bench).all_tags() )

Using the OSM ID you also have access to `b6.osm_way_id` for finding a ways (areas and lines), and for finding relations you have `b6.osm_relation_area_id`, `b6.osm_relation_id`

For search you can always use `b6.find` but if you are being specific, aside from `b6.find_point` and `b6.find_area` there is also `b6.find_path`. More on this below. 


In [ ]:
# If you didn't know what other features were in the park, 
# you could limit your search to what is in the park area
contained_in_oakleysq_gardens = b6.find( b6.intersecting( b6.find_feature(oakleysq_gardens) ) )

for osmid, feature in w(contained_in_oakleysq_gardens):
    print( f"{osmid}\n{feature.all_tags()}" )

---
Note that searching for what is in the park yields different types of features. 

- It yields `points`, which in this instance are benches. 
- It yields `paths` and `areas`   
- It also yields `access-path` which is also a `path` - these are paths we have automatically generated to ensure all features are connected to the network. This is helpful, for example if you want to compute routes.

In [ ]:
# you could be more precise and only search for `points` within the park
points_in_oakleysq_gardens = b6.find_points( b6.intersecting( b6.find_feature(oakleysq_gardens) ) )

for osmid, point in w(points_in_oakleysq_gardens):
    print( f"{osmid}\n{point.all_tags()}" )

In [ ]:
# you could be more precise and only search for `areas` within the park
areas_in_oakleysq_gardens = b6.find_areas( b6.intersecting( b6.find_feature(oakleysq_gardens) ) )

for osmid, area in w(areas_in_oakleysq_gardens):
    print( f"{osmid}\n{area.all_tags()}" )

In [ ]:
# you could be more precise and only search for `paths` within the park
paths_in_oakleysq_gardens = b6.find_paths( b6.intersecting( b6.find_feature(oakleysq_gardens) ) )

for osmid, path in w(paths_in_oakleysq_gardens):
    print( f"{osmid}\n{path.all_tags()}" )

##  Filter

You can use `.filter` to narrow down your search.
You can also then apply `.map` to grab the value of a tag.
When grabbing the value of a tag you have four options:
- `get` which gets you whatever the value is
- `get_string`
- `get_int`
- `get_float`

In [ ]:
# you can search for features and filter to only get ones that contain a certain tag
contained_in_oakleysq_gardens = b6.find( b6.intersecting( b6.find_feature(oakleysq_gardens) ) )
amenities_in_oakleysq_gardens = contained_in_oakleysq_gardens.filter( b6.keyed("#amenity") )

for osmid, amenity in w(amenities_in_oakleysq_gardens):
    print( f"{osmid}\n{amenity.all_tags()}" )

In [ ]:
# more generally you can do a wide search, using the searchable tags,
# then do a filter for features that have other non-searchable tags, present. 
offices_with_levels = b6.find_areas( b6.tagged("#building","office") ) \
                          .filter( b6.keyed("building:levels") )

for osmid, office in w(offices_with_levels):
    print( f"{osmid}\n{office.all_tags()}" )

In [ ]:
# even more fun, you can extract the value of that tag
levels_in_offices = offices_with_levels.map( lambda b: b6.get_int(b, "building:levels") )

for osmid, levels in w(levels_in_offices):
    print( f"{osmid} -->{levels} levels" )

In [ ]:
# you can use that to then analyse/categorise/filter by value as you need, using python
df = pd.DataFrame(w(levels_in_offices),
                  columns =["office","number_of_levels"]
                 ).set_index("office")

df.plot.hist()

## More specific search areas

In [ ]:
# you can search for features within some distance, as the crow flies, of a [latitude,longitude] 
distance = 100 # in metres
nearby_oakleysq_gardens = b6.find( b6.within_cap(b6.ll(*OAKLEY_SQUARE_GARDENS_LATLON),
                                                 distance) )

for osmid, feature in w(nearby_oakleysq_gardens):
    print( f"{osmid}\n{feature.all_tags()}" )

The idea of `cap` comes from the s2 library, where `cap` is a key type (https://pkg.go.dev/github.com/golang/geo/s2#Cap)
and it is the result of passing a 2d plane through the sphere.

In [ ]:
# you can also search for features within some distance,
# via some mode, say walking, of a feature in the data
distance = 100 # in metres
oakleysq_gardens = b6.find_area( b6.osm_way_area_id(OAKLEY_SQUARE_GARDENS_WAY_ID) )

nearby_oakleysq_gardens = oakleysq_gardens.reachable("walk",
                                                     distance,
                                                     b6.all()
                                                    )

for osmid, feature in w(nearby_oakleysq_gardens):
    print( f"{osmid}\n{feature.all_tags()}" )

You can also try `bus` and `car` for different modes, as input to `reachable`
If you were only after reachable features with particular tags, you could also use `b6.keyed` to find the features with that key or `b6.tagged` to find the features with that key:tag pair. 

In [ ]:
# you can search within any polygon by defining that polygon as a wkt
oakleysq_gardens_polygon = b6.wkt(OAKLEY_SQUARE_GARDENS_POLYGON_WKT)
contained_in_oakleysq_gardens = b6.find( b6.intersecting(oakleysq_gardens_polygon) )

for osmid, feature in w(contained_in_oakleysq_gardens):
    print( f"{osmid}\n{feature.all_tags()}" )